In [8]:
!pip install sentencepiece
!pip install -U sentence-transformers
!pip install PyPDF2
!pip install transformers
!pip install huggingface_cli
!pip install accelerate
!pip install huggingface_hub
!pip install bitsandbytes

ERROR: Could not find a version that satisfies the requirement huggingface_cli (from versions: none)
ERROR: No matching distribution found for huggingface_cli


In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /roo

In [9]:
from sentence_transformers import SentenceTransformer, util, CrossEncoder
from PyPDF2 import PdfReader

from transformers import AutoTokenizer
import transformers
import torch

# base LLM model
llm_model = "meta-llama/Llama-2-7b-chat-hf"

# sentence transformer
model = SentenceTransformer('all-MiniLM-L6-v2')
cross_encoder = CrossEncoder("sentence-transformers/msmarco-MiniLM-L6-cos-v5")

# pdf reader
reader = PdfReader('1709.04396.pdf')

# tokenizer
tokenizer = AutoTokenizer.from_pretrained(llm_model)

# llm text generation pipeline
pipeline = transformers.pipeline(
    "text-generation",
    model=llm_model,
    torch_dtype=torch.float16,
    device_map="auto",
)

AttributeError: ignored

In [ ]:
def generate_embeddings():
  # compile pages and generate text
  pages = []
  for page in reader.pages:
    pdf_text = page.extract_text()
    pages.append(pdf_text)

  # join all pages
  pages =" ".join(pages)

  # convert to tokens
  tokens = pages.split()

  # parameters for paragraphing
  step_size = 100
  window_size = 50
  sentences = []

  # generate sentences
  for i in range(0, len(tokens), step_size):
    window = tokens[i:i+window_size]
    if len(window) < window_size:
      break

    sentences.append(window)

  # convert to paragraphs
  paragraphs = [" ".join(s) for s in sentences]


  # generate paragraph embeddings
  embeddings = model.encode(paragraphs, show_progress_bar=True)
  return embeddings, paragraphs

In [ ]:
def search_answers(question, embeddings, paragraphs):
  questionEmbedding = model.encode(question)
  search_results = util.semantic_search(
      questionEmbedding,
      embeddings,
      top_k=35
  )

  # generate pairs for cross encoder
  qa_pair = []
  for result in search_results[0]:
    passage = paragraphs[result['corpus_id']]
    question = question
    qa_pair.append([passage, question])

  # compute question, paragraph similarity
  scores = cross_encoder.predict(qa_pair)

  # append scores in search_results
  for i in range(len(scores)):
    search_results[0][i]['cross_score'] = scores[i]

  # sort search results
  search_results = sorted(search_results[0], key=lambda x: x['cross_score'], reverse=True)

  # pick top 5 paragraphs
  top_5 = []
  for result in search_results[:5]:
    result_text = paragraphs[result['corpus_id']]
    result_text = result_text.replace("\n", ' ')
    top_5.append(result_text)

  return top_5

In [ ]:
def generate_response(paragraphs,query):
  prompt = f"""<s>[INST]
Given below are some paragraphs that contains context for the asked question. Your job is to answer the question that is asked using the paragraphs
Paragraph 1: {paragraphs[0]}
Paragraph 2: {paragraphs[1]}
Paragraph 3: {paragraphs[2]}
Paragraph 4: {paragraphs[3]}
Paragraph 5: {paragraphs[4]}
[/INST]
{query}
  """.strip()
  sequences = pipeline(
      prompt,
      do_sample=True,
      top_k=10,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
  )

  return sequences[0]['generated_text']

In [ ]:
def answer_question(question, embeddings, paragraphs):
  top_5_responses = search_answers(
      question,
      embeddings,
      paragraphs
  )
  response = generate_response(top_5_responses,question)
  print(response)

In [ ]:
embeddings, paragraphs = generate_embeddings()

In [ ]:
question = input("Enter your question: ")
answer_question(question, embeddings, paragraphs)